# Cvičenie 10: Fuzzy systémy

Fuzzy logika je metodológia, ktorá vychádza z predpokladu, že tvrdenia nemusia byť iba úplne pravdivé alebo nepravdivé, ale ich pravdivosť je možné vyjadriť na škále. Fuzzy logika teda nepracuje s bežnými premennými, používa ale fuzzy premenné, ktoré majú definované univerzum (všetky mozné hodnoty alebo rozptyl) a aktuálno hodnotu. Ak rozptyl je nulový, fuzzy premenná sa stane obyčajným premenným s reálnou hodnotou.

Fuzzy premenná môže byť takisto vyjadrená výrazom, ktorý je zvyčajne ľudský koncept (prídavné meno alebo príslovka). Takéto výrazy vieme sformalizovať vďaka fuzzy množinám, ktoré reálnym hodnotám prideľujú aj mieru príslušnosti. Napríklad, ak máme fuzzy premennú, ktorá vyjadruje **rýchlosť vozidla**, táto premenná môže byť vyjadrená výrazom **rýchle vozidlo**, kde skutočná rýchlosť 200 km/h môže mať príslušnosť 1.0 ("úplne rýchle vozidlo") a skutočná hodnota 80 km/h príslušnosť 0.1 ("ledva rýchle vozidlo").

Pomocou fuzzy logiky vieme teda vytvoriť fuzzy riadiaci systém, ktorý ovláda ľudské koncepty akosti a miery. Ako každý riadiaci systém, aj fuzzy systémy obsahujú **pravidlá**, ktoré majú tvar *IF podmienka THEN dôsledok* podobne ako v experných systémoch. Jediný rozdiel je, že podmienky aj dôsledky obsahujú fuzzy premenné.

Inferenciu vo fuzzy systémoch vieme rozdeliť na tri kroky:
1. fuzzifikácia - vstupné reálne hodnoty premeníme na fuzzy hodnoty
2. inferencia na základe tabuľky pravidiel - samotný regulačný algoritmus, funguje obdobne ako pri expertných systémoch
3. defuzzifikácia - inferencia nám dáva výsledok ako fuzzy premennú, jej hodnotu prevedieme na reálne číslo alebo na očakávaný tvar

## Úloha 1: Definícia fuzzy systému v Pythone

Ako aj pre ostatné štandardné metódy umelej inteligencie, aj pre fuzzy systémy existujú knižnice, ktoré implementujú základnú logiku takýchto systémov. Našou úlohou tým pádom je iba samotná definícia pravidiel a následné zadanie vstupných hodnôt do výpočtov, nemusíme sa starať o implementáciu riadiacej logiky. Na dnešnom cvičení použijeme knižnicu `scikit-fuzzy`, ktorá ale nie je súčasťou inštalácie Pythonu alebo Anacondy, potrebujeme ju doinštalovať pomocou príkazu v príkazovom riadku alebo v Anaconda Prompt

```pip install scikit-fuzzy```

alebo

```conda install scikit-fuzzy```.

Ak používate grafické IDE na programovanie, napr. PyCharm alebo Visual Studio Code, tieto zvyčajne majú grafické rozhranie aj pre inštaláciu knižníc.

Úspešnosť inštalácie otestujete v príkazovom riadku zadaním príkazu `pip list` - zobrazí sa vám zoznam všetkých inštalovaných knižníc, medzi nimi aj `scikit-fuzzy`. Alternatívne môžete spustiť skript, v ktorom naimportujete knižnicu príkazom `import skfuzzy`.

### 1.1 Import potrebných knižníc

Okrem knižnice `scikit-fuzzy` potrebujeme ešte aj ďalšie knižnice, ktoré naimportujeme na začiatku nášho skriptu.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

Ak sa vám nepodarý importovať niektorú knižnicu, pretože chýba, viete ju doinštalovať pomocou príkazu `pip`. Knižnicu `matplotlib` použijeme na vizualizáciu niektorých častí nášho systému, `numpy` je šikovná knižnica, ktorá nám poskytuje C-čkovú implementáciu polí v Pythone (použijeme ju pri definícii fuzzy množín).

### 1.2 Definícia premenných

Ak chceme vytvoriť fuzzy systém, prvým krokom musí byť definícia pravidiel, ktoré takisto potrebujeme rozdeliť na predpoklady a dôsledky, ku ktorým potrebujeme nevyhnutne aj fuzzy premenné. Na dnešnom cvičení vytvoríme veľmi jednoduchý fuzzy systém s tromi pravidlami. Tento systém vám má pomôcť pri hodnotení predmetov pri koncoročnom dotazníku na MAISe. Predpokladáme, že hodnotenie predmetu závisí iba od vašej spokojnosti s prednáškami a s cvičeniami.

Zadefinujeme teda nasledovné premenné:

* vstupy
  * kvalita cvičení
    * univerzum - hodnoty od 0 po 10
    * fuzzy hodnoty - slabá, priemerná, dobrá
  * kvalita prednášok
    * univerzum - hodnoty od 0 po 10
    * fuzzy hodnoty - slabá, priemerná, dobrá
* výstupy
  * hodnotenie predmetu
    * univerzum - hodnoty od 0 po 10
    * fuzzy hodnoty - nízke, priemerné, vysoké

Platí, že vstupné premenné patria medzi predpoklady (definujú ich), kým výstupné premenné použijeme pre dôsledky.

#### 1.2.1 Definícia predpokladov

Predpoklady (po anglicky *antecedent*) definujeme pomocou triedy `Antecedent` z modulu `skfuzzy.control` a to tak, že definujeme univerzum (ako `numpy` pole) a zadáme label pre ďalšie použitie premennej (hodnoty sa priradzujú na základe labelu). Podrobnejšie informácie nájdete [v dokumentácii](https://pythonhosted.org/scikit-fuzzy/api/skfuzzy.control.html#antecedent).

Najprv zadefinujeme premennú pre vyjadrenie kvality cvičení:

In [ ]:
labs_quality = ctrl.Antecedent(np.arange(0, 11, 1), 'labs')

Funkcia `numpy.arange` nám vygeneruje interval od 0 po 10 (11 už neberie, podobne ako slicing v čistom Pythone) s krokom 1. Následne definujeme label pre premennú.

Predpoklad je síce už pripravený, ale potrebujeme ešte zadefinovať fuzzy hodnoty, resp. fuzzy množiny. Tieto vieme urobiť manuálne, alebo môžeme ich vygenerovať automaticky. Pre predpoklady využijeme práve možnosť automatického generovania, na čo slúži metóda `automf` (*automatic membership function*). Ako parameter uvedieme počet kategórií (môže nadobudnúť hodnoty 3, 5 alebo 7 - ak chcete iný počet, musíte zadať aj zoznam labelov jednotlivých hodnôt).

In [ ]:
labs_quality.automf(3)

Následne vieme vizualizovať fuzzy hodnoty tejto premennej a to pomocou metódy `view`. Na vykresľovanie sa použije knižnica `matplotlib`, z ktorej zavoláme metódu `show` aby graf bol zobrazený na dlhší čas.

In [ ]:
labs_quality.view()
plt.show()

Kód nám vykreslí nasledujúci graf:

![Lab quality variable](figures/lab10/10.1-lab-quality-variable.png)

Premennú pre kvalitu prednášok zadefinujeme podobne:

In [ ]:
lecture_quality = ctrl.Antecedent(np.arange(0, 11, 1), 'lectures')
lecture_quality.automf(3)

#### 1.2.2 Definícia dôsledku

V dôsledku použijeme iba jednu premennú, a tá vyjadruje hodnotenie predmetu. Základná definícia je podobná ako v prípade vstupných premenných, avšak použijeme triedu `Consequent` ([viď dokumentácia](https://pythonhosted.org/scikit-fuzzy/api/skfuzzy.control.html#consequent)). Fuzzy hodnoty teraz zadefinujeme manuálne. Knižnica definuje niekoľko funkcií príslušnosti ([viď dokumentácia](https://pythonhosted.org/scikit-fuzzy/api/skfuzzy.html)), my ale použijeme základnú trojuholníkovú. V štandardnej fuzzy logike sa použije Gaussova funkcia, ktorú ale veľmi dobre vieme nahradiť trojuholníkovou funkciou, ktorá navyše lepšie podporuje základné matematické operácie.

In [ ]:
rating = ctrl.Consequent(np.arange(0, 11, 1), 'rating')

Následne zadefinujeme fuzzy hodnoty *nízke*, *priemerné* a *vysoké* podobne ako v prípade vstupných premenných:

In [ ]:
rating['low'] = fuzz.trimf(rating.universe, [0, 0, 5])
rating['medium'] = fuzz.trimf(rating.universe, [0, 5, 10])
rating['high'] = fuzz.trimf(rating.universe, [5, 10, 10])

Metóda [`trimf`](https://pythonhosted.org/scikit-fuzzy/api/skfuzzy.html#skfuzzy.trimf) (*triangular membership function*) zadefinuje trojuholníkovú funkciu príslušnosti, pre ktorú musíme určiť množinu všetkých hodnôt, teda univerzum príslušnej premennej a následne odovzdávame aj zoznam s tromi hodnotami [a, b, c]: najnižšia hodnota, hodnota s najvyššou mierou príslušnosti, najvyššia hodnota, pričom platí *a <= b <= c*.

Po vizualizácii dostaneme graf podobný ako pri vstupnej premennej, ale s nami definovanými labelmi:

![Rating variable](figures/lab10/10.2-rating-variable.png)

### 1.3 Definícia pravidiel

Pre náš fuzzy systém zadefinujeme tri pravidlá, ktoré nám budú určovať hodnotu výstupnej premennej, teda dôsledok:

1. AK kvalita cvičení je slabá ALEBO kvalita prednášok je slabá POTOM hodnotenie predmetu je nízke
2. AK kvalita cvičení je priemerná POTOM hodnotenie predmetu je priemerné
3. Ak kvalita cvičení je dobrá A kvalita prednášok je dobrá POTOM hodnotenie predmetu je vysoké

Všimnite si, že v týchto pravidlách pracujeme s fuzzy hodnotami, nikde nefigurujú reálne čísla, tie použijeme až pri riešení konkrétneho príkladu.

Pravidlá zadefinujeme pomocou triedy `Rule` z modulu `skfuzzy.control` ([viď dokumentácia](https://pythonhosted.org/scikit-fuzzy/api/skfuzzy.control.html#rule)). Každé pravidlo musí mať určený predpoklad a dôsledok, pričom vieme spojiť viaceré predpoklady alebo dôsledky pomocou štandardných operácií A ZÁROVEŇ (`&`), ALEBO (`|`) a NEGÁCIA (`~`). Aj k pravidlám vieme priradiť label, ale nie je to nevyhnutné.

Samotné pravidlá zadefinujeme nasledovne:

In [ ]:
rule1 = ctrl.Rule(
    labs_quality['poor'] | lecture_quality['poor'],
    rating['low'])

In [ ]:
rule2 = ctrl.Rule(
    labs_quality['average'],
    rating['medium'])

In [ ]:
rule3 = ctrl.Rule(
    lecture_quality['good'] & labs_quality['good'],
    rating['high'])

### 1.4 Definícia fuzzy systému

Ak máme pripravené všetky premenné a pravidlá, vieme zadefinovať celý inferenčný systém pomocou triedy `ControlSystem` z modulu `skfuzzy.control` ([pozri dokumentáciu](https://pythonhosted.org/scikit-fuzzy/api/skfuzzy.control.html#controlsystem)). Trieda má iba jeden parameter, a to zoznam všetkých pravidiel.

Keďže náš systém funguje na základe troch vyššie definovaných pravidiel, vytvoríme ho nasledovne:

In [ ]:
rating_ctrl = ctrl.ControlSystem([rule1, rule2, rule3])

### 1.5 Testovanie a použitie fuzzy systému

Ak systém je pripravený, na jeho použitie potrebujeme vytvoriť simulácie, t.j. inštancie všeobecného riešiteľa - samotného systému. V našom prípade by sme vytvorili osobitné simulácie pre hodnotenie rôznych predmetov.

Simuláciu vytvoríme pomocou triedy `ControlSystemSimulation` z modulu `skfuzzy.control` ([viď dokumentácia](https://pythonhosted.org/scikit-fuzzy/api/skfuzzy.control.html#controlsystemsimulation)). Pre vytvorenie simulácie potrebujeme určiť fuzzy inferenčný systém (konštruktor má aj ďalšie parametre, tie ale nie sú povinné):

In [ ]:
my_rating = ctrl.ControlSystemSimulation(rating_ctrl)

Nezostáva nám nič iné ako zadefinovať hodnoty vstupných premenných ako reálne čísla:

In [ ]:
my_rating.input['labs'] = 6.5
my_rating.input['lectures'] = 9.8

V tomto prípade cvičenia sa nám zdajú byť iba trošku lepšie ako priemer, kým s prednáškami sme skoro úplne spokojní. Vidíme, že na základe týchto dvoch čísel by sme ťažko vyjadrili našu spokojnosť s celým predmetom, urobili by sme tak intuitívne. Ale práve vďaka fuzzy logike a zadefinovaným pravidlám túto hodnotu vypočíta náš inferenčný systém ak zavoláme metódu `compute` pre simuláciu:

In [ ]:
my_rating.compute()

Následne vieme vypísať vypočítanú hodnotu:

In [ ]:
print("The suggested rating is {:.2f}".format(my_rating.output['rating']))

Alebo uvažovanie vieme aj vizualizovať pomocou inferenčného systému:

In [ ]:
rating.view(sim=my_rating)
plt.show()

Kde výsledok vyzerá nasledovne:

![Fuzzy inference result](figures/lab10/10.3-fuzzy-result.png)

Na grafe vidíme, že naše hodnotenie najviac patrí do kategórie *priemerné*, avšak čiastočne ho môžeme vnímať ako fuzzy hodnotu *vysoké*.

### 1.6 Úprava fuzzy systému

Vyskúšajte funkčnosť fuzzy systému na ďalších vstupoch a upravte aj niektoré premenné alebo pravidlá aby ste zistili, ako tieto objekty ovplyvňujú výsledok systému.

## Úloha 2: Bonusová úloha

Vytvorte pomocou knižnice `scikit-fuzzy` ďalší fuzzy inferenčný systém, pre ktorý zadefinujte minimálne 5 fuzzy premenných (predpokladov a dôsledkov) a minimálne 10 pravidiel. Do vášho kódu pridajte aj formálnu definíciu premenných a pravidiel (ako v bodoch 1.2 a 1.3) do komentárov. Ak vaše riešenie (Python skript) pošlite cez e-mail do 24. 4. 2020 (do 24:00), môžete získať bonusové body - max. 2 za jednoduchý systém, max. 5 ak systém bude obsahovať pravidlá, kde premenná v dôsledku jedného pravidla bude premennou v predpoklade druhého pravidla.

**Poznámka:** riešenie cvičenia ako skript [nájdete tu](solutions/lab10_solution.py).